In [1]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
import sys
# import booleanNetwork module from ./src
from BNMPy import booleanNetwork as bn
from BNMPy import BMatrix  

In [2]:
file = 'input_files/dnmt3a_equations.txt'

In [3]:
with open(file) as f:
    file_data = f.read()

In [4]:
network = BMatrix.load_network_from_string(file_data)

No initial state provided, using a random initial state


In [12]:
# 3 inputs: FLT3, DNMT3A, NPM1
# wild-type
network.setInitialValue('FLT3', 0)
network.setInitialValue('NPM1', 1)
network.setInitialValue('DNMT3A', 1)
results = network.update(20)
results = results[-1, :]
results = {k: results[v] for k, v in network.nodeDict.items()}

In [13]:
results['APOPTOSIS'] = results['TP53'] - results['BCL2']
results['DIFFERENTIATION'] = results['CEBPA'] + results['ETV6'] - results['MEIS1']
results['PROLIFERATION'] = results['MYC'] + results['CCND1'] + results['SOX4'] + results['MEIS1'] + results['STAT5A']

# calculate the network scores
results['NETWORK'] = results['PROLIFERATION'] - results['APOPTOSIS'] - results['DIFFERENTIATION']

In [14]:
print(results['APOPTOSIS'], results['DIFFERENTIATION'], results['PROLIFERATION'], results['NETWORK'])

1 2 0 -3


In [17]:
network.knockout('FLT3', 1)

In [18]:
results = network.update(20)
results = results[-1, :]
results = {k: results[v] for k, v in network.nodeDict.items()}
results['APOPTOSIS'] = results['TP53'] - results['BCL2']
results['DIFFERENTIATION'] = results['CEBPA'] + results['ETV6'] - results['MEIS1']
results['PROLIFERATION'] = results['MYC'] + results['CCND1'] + results['SOX4'] + results['MEIS1'] + results['STAT5A']

# calculate the network scores
results['NETWORK'] = results['PROLIFERATION'] - results['APOPTOSIS'] - results['DIFFERENTIATION']
print(results['APOPTOSIS'], results['DIFFERENTIATION'], results['PROLIFERATION'], results['NETWORK'])

1 0 2 1


In [20]:
network.undoKnockouts()
network.setInitialValue('FLT3', 0)
network.setInitialValue('NPM1', 1)
network.setInitialValue('DNMT3A', 1)
network.knockout('NPM1', 0)
results = network.update(20)
results = results[-1, :]
results = {k: results[v] for k, v in network.nodeDict.items()}
results['APOPTOSIS'] = results['TP53'] - results['BCL2']
results['DIFFERENTIATION'] = results['CEBPA'] + results['ETV6'] - results['MEIS1']
results['PROLIFERATION'] = results['MYC'] + results['CCND1'] + results['SOX4'] + results['MEIS1'] + results['STAT5A']

# calculate the network scores
results['NETWORK'] = results['PROLIFERATION'] - results['APOPTOSIS'] - results['DIFFERENTIATION']
print(results['APOPTOSIS'], results['DIFFERENTIATION'], results['PROLIFERATION'], results['NETWORK'])

0 1 1 0
